# **Phase 2 - Milestone 2**

**Ediashta Revindra Amirussalam**
<br>FTDS-020-RMT

## Objective

## 00 - Import Libraries

In [25]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import random

## 01 - Data Loading

In [4]:
main_path = ('./datasets/')
os.listdir(main_path)

['test', 'train']

In [7]:
train_dir = os.path.join(main_path, 'train')
test_dir = os.path.join(main_path, 'test')
os.listdir(train_dir)

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [18]:
def check_dir(dir=None, labels=None):
  i=0
  for label in labels:
    num_data= len(os.listdir(os.path.join(dir, label)))
    print(f'Data', label, ' :', num_data)
    i+= num_data
  print('Total Data :', i)
    


In [19]:
print('train set: \n' + '='*50)
check_dir(train_dir,os.listdir(train_dir))
print('\ntest set: \n' + '='*50)
check_dir(test_dir,os.listdir(test_dir))

train set: 
Data angry  : 3995
Data disgusted  : 436
Data fearful  : 4097
Data happy  : 7215
Data neutral  : 4965
Data sad  : 4830
Data surprised  : 3171
Total Data : 28709

test set: 
Data angry  : 958
Data disgusted  : 111
Data fearful  : 1024
Data happy  : 1774
Data neutral  : 1233
Data sad  : 1247
Data surprised  : 831
Total Data : 7178


## 02 - EDA

In [35]:
train_angry= glob.glob(train_dir + '/angry/*.png')
train_disgusted= glob.glob(train_dir + '/disgusted/*.png')
train_fearful= glob.glob(train_dir + '/fearful/*.png')
train_happy= glob.glob(train_dir + '/happy/*.png')
train_neutral= glob.glob(train_dir + '/neutral/*.png')
train_sad= glob.glob(train_dir + '/sad/*.png')
train_surprised= glob.glob(train_dir + '/surprised/*.png')

test_angry= glob.glob(test_dir + '/angry/*.png')
test_disgusted= glob.glob(test_dir + '/disgusted/*.png')
test_fearful= glob.glob(test_dir + '/fearful/*.png')
test_happy= glob.glob(test_dir + '/happy/*.png')
test_neutral= glob.glob(test_dir + '/neutral/*.png')
test_sad= glob.glob(test_dir + '/sad/*.png')
test_surprised= glob.glob(test_dir + '/surprised/*.png')


In [38]:
def random_sample(image_files, num_samples=5):
  random_files= random.sample(image_files, num_samples)
  return random_files

In [69]:
def visualize_img(random_data, fig, axes, loc, labels=None):
  for i, image_path in enumerate(random_data):
    image= plt.imread(image_path)
    axes[loc, i].imshow(image, cmap='gray') # bisa tanpa cmap kalau bener2 RGB
    axes[loc, i].set_title(f'{labels}, {image.shape}')
    axes[loc, i].axis('off')

In [ ]:
count = 0
fig, axes= plt.subplots(7,5, figsize=(15,12))

for i in os.listdir(train_dir):
    visualize_img(
        random_sample(
            glob.glob(train_dir + '/' + i +'/*.png')
        ), fig, axes, count, i
    )
    count+=1

plt.show()